In [871]:
import pandas as pd
import numpy as np
import os 

In [872]:
cwd = os.getcwd()

df = pd.read_excel(cwd+"/B2B Data Provider-sample1.xlsx", index_col=0)

#replace all Nan values with -
df = df.replace(np.nan, "-")

In [873]:
print(df.columns)

Index(['Starting Price - Monthly', 'Starting Price - Annually',
       'Pricing Range', 'Pricing comments', 'Additional Pricing Info',
       'Website', 'Contract', 'Solutions', 'Unlimited Data Plan',
       'CRM Integrations', 'SEPI', 'SalesTeamType', 'Free Trial',
       'ContactEnrichment', 'IntentData', 'Other', 'location'],
      dtype='object')


In [874]:
#removing unnecessary columns
df_soft = df.drop(['Additional Pricing Info','Website','Unlimited Data Plan', 'Pricing Range', 'Pricing comments','Free Trial','Other'], axis=1)

#drop the pricing from data and this can be used to create a feature matrix
df_pro = df_soft.drop(['Starting Price - Monthly', 'Starting Price - Annually'], axis=1)
df_pro

,Contract,Solutions,CRM Integrations,SEPI,SalesTeamType,ContactEnrichment,IntentData,location
Company,,,,,,,,
ZoomInfo,Annual,"email,workphone,mobile","Microsoft Dynamics,HubSpot,Salesforce,Pipedriv...","Outreach,Salesloft","SMB,MidMarket,Enterprise",Yes,Yes,USA/NorthAmerica
Cognism,Annual,"email,workphone,mobile","Salesforce,HubSpot,Pipedrive,Microsoft Dynamics",Salesloft,"SMB,MidMarket,Enterprise",Yes,Yes,International
Uplead,Annual or Monthly,"email,workphone,mobile","Salesforce,HubSpot,Insightly,Pipedrive,Zoho,Mi...","Outreach,Salesloft,Reply,Woodpecker,Lemlist,Close","SMB,MidMarket,Enterprise",Yes,Yes,USA/NorthAmerica
Lead411,Annual or Monthly,"email,workphone,mobile","Salesforce,HubSpot,Pipedrive,Microsoft Dynamic...",-,"SMB,MidMarket,Enterprise",Yes,Yes,USA/NorthAmerica
Apollo,Annual or Monthly,"email,workphone,mobile","Salesforce,HubSpot","Salesloft,Outreach","SMB,MidMarket,Enterprise",Yes,No,USA/NorthAmerica
Seamless,Annual,"email,workphone,mobile","Salesforce,HubSpot,Microsoft Dynamics,Keap,Zoh...","Salesloft,Outreach","SMB,MidMarket",Yes,No,USA/NorthAmerica
RocketReach,Annual or Monthly,"email,workphone,mobile","Salesforce,HubSpot","Salesloft,Outreach","SMB,MidMarket",No,No,USA/NorthAmerica
Lusha,Annual or Monthly,"email,workphone","Salesforce,HubSpot","Salesloft,Outreach",SMB,Yes,No,USA/NorthAmerica
Crunchbase,Annual or Monthly,email,Salesforce,-,"SMB,MidMarket,Enterprise",No,No,USA/NorthAmerica


In [875]:
#convert all the contract to list values
df_pro['Contract'] = df_pro['Contract'].str.replace(" or ", ",").str.lower().str.split(',')
#convert all solutions values to list 
df_pro['Solutions'] = df_pro['Solutions'].str.lower().str.split(",")
#convert all the CRM values to list
df_pro['CRM Integrations'] = df_pro['CRM Integrations'].str.lower().str.split(",")
#convert all SEPI values to list
df_pro['SEPI'] = df_pro['SEPI'].str.lower().str.split(",")
#convert all location values to list
df_pro['location'] = df_pro['location'].str.lower().str.split(",")
#convert all location values to list
df_pro['SalesTeamType'] = df_pro['SalesTeamType'].str.lower().str.split(",")

In [876]:
def get_software_data_matrix(dataframe):

    for i, col in enumerate(dataframe.columns):

        for index,row in dataframe.iterrows():
            for contract in row[col]:
                if (dataframe.at[index,col] == "Yes") or (dataframe.at[index,col] == "No"):
                    pass
                else:
                    dataframe.at[index,contract] =1

    #replace the Yes and no values to 1 and 0 respectively for ContactEnrichment and IntentData columns
    dataframe = dataframe.replace('Yes', 1.0)
    dataframe = dataframe.replace('No', 0)


    #now drop all the initial columns that is not a matrix
    dataframe =dataframe.drop(['Contract','Solutions','CRM Integrations','SEPI','location', '-', 'SalesTeamType'], axis=1)

    #lastly replace all the nan values with 0
    dataframe= dataframe.replace(np.nan, 0)

    #lower casing all the column names
    dataframe.columns = dataframe.columns.str.lower()

    return dataframe

df_pro = get_software_data_matrix(df_pro)
df_pro

,contactenrichment,intentdata,annual,monthly,email,workphone,mobile,microsoft dynamics,hubspot,salesforce,...,salesloft,reply,woodpecker,lemlist,close,smb,midmarket,enterprise,usa/northamerica,international
Company,,,,,,,,,,,,,,,,,,,,,
ZoomInfo,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
Cognism,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
Uplead,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
Lead411,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
Apollo,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
Seamless,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
RocketReach,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
Lusha,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
Crunchbase,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0


In [877]:
# #set the contract first
# for index,row in df_pro.iterrows():
#     for contract in row['Contract']:
#         df_pro.at[index,contract] =1

# #set the solutions 
# for index,row in df_pro.iterrows():
#     for sol in row['Solutions']:
#         df_pro.at[index,sol] =1
    
# #set the CRM
# for index,row in df_pro.iterrows():
#     for crm in row['CRM Integrations']:
#         df_pro.at[index,crm] = 1
        
# #set the Sales engagement matrix
# for index,row in df_pro.iterrows():
#     for sepi in row['SEPI']:
#         df_pro.at[index,sepi] = 1
        

# #set the location matric
# for index,row in df_pro.iterrows():
#     for loc in row['location']:
#         df_pro.at[index,loc] = 1
        
# #set the location matric
# for index,row in df_pro.iterrows():
#     for sales in row['SalesTeamType']:
#         df_pro.at[index,sales] = 1
        
# #now drop all the initial columns that is not a matrix
# df_pro =df_pro.drop(['Contract','Solutions','CRM Integrations','SEPI','location', '-', 'SalesTeamType'], axis=1)

# #lastly replace all the nan values with 0
# df_pro= df_pro.replace(np.nan, 0)


# #lower casing all the column names
# df_pro.columns = df_pro.columns.str.lower()

In [878]:
#check the shape
df_pro.shape

(10, 35)

In [879]:
#treated dataframe 
df_pro.columns

Index(['contactenrichment', 'intentdata', 'annual', 'monthly', 'email',
       'workphone', 'mobile', 'microsoft dynamics', 'hubspot', 'salesforce',
       'pipedrive', 'sugarcrm', 'zoho', 'insightly', 'copper', 'nimble',
       'infusionsoft', 'capsule', 'nutshell', 'base', 'vtiger', 'keap',
       'freshsales', 'freshworks', 'outreach', 'salesloft', 'reply',
       'woodpecker', 'lemlist', 'close', 'smb', 'midmarket', 'enterprise ',
       'usa/northamerica', 'international'],
      dtype='object')

**Now the idea is to cross multiply the values user values with and find the weighted values**

## USER data

In [924]:
df_user = pd.read_csv("user_data.csv", index_col=0)
df_user

,data,contract,price,contacts,duration,Location,features,salesTeam,crm,SEPI_tools
0,['email'],['Monthly'],NaN,More than 5000,['Flexible'],['USA/NorthAmerica'],"['ContactEnrichment', 'IntentData']",['MidMarket'],"['Freshsales', 'SugarCRM', 'Zoho']",['Salesloft']


In [925]:
df_user.dtypes


data           object
contract       object
price         float64
contacts       object
duration       object
Location       object
features       object
salesTeam      object
crm            object
SEPI_tools     object
dtype: object

In [926]:
for i, col in enumerate(df_user.columns):
    if df_user[col].dtype == "int64":
        pass
    else:
        if col == "Location":
            if df_user['Location'].item().replace("'", "").strip("[]") == "Global":
                df_user['Location'] = [['usa/northamerica', 'international']]
            else:
                df_user[col] = df_user[col].item().replace("'", "").strip("[]").lower()
                df_user[col] = df_user[col].str.split(",")
        else:
            df_user[col] = df_user[col].item().replace("'", "").strip("[]").lower()
            df_user[col] = df_user[col].str.split(",")
        
        
    

AttributeError: 'float' object has no attribute 'replace'

In [927]:
df_user

,data,contract,price,contacts,duration,Location,features,salesTeam,crm,SEPI_tools
0,[email],[monthly],NaN,More than 5000,['Flexible'],['USA/NorthAmerica'],"['ContactEnrichment', 'IntentData']",['MidMarket'],"['Freshsales', 'SugarCRM', 'Zoho']",['Salesloft']


In [884]:
def get_user_data_matrix(dataframe):

    for i, col in enumerate(dataframe.columns):
        if col not in ['price','contacts']:
            if dataframe[col][0][0] == '"i dont have any crm"' or dataframe[col][0][0] == '"i dont have a sales engagement platform"':
                pass
            else:
                if col in ['crm', 'SEPI_tools']:
                    total_tools = len(dataframe['SEPI_tools'].item()) + len(dataframe['crm'].item())
                    for index,row in dataframe.iterrows():
                        for contract in row[col]:
                            dataframe.at[index,contract] = 1/total_tools
                else:
                    for index,row in dataframe.iterrows():
                        for contract in row[col]:
                            dataframe.at[index,contract] =1
            

    #now drop all the initial columns that is not a matrix
    dataframe = dataframe.iloc[:,10:]

    #lastly replace all the nan values with 0
    dataframe= dataframe.replace(np.nan, 0)

    #lower casing all the column names
    dataframe.columns = dataframe.columns.str.lower()
    
    #lower casing all the column names
    dataframe.columns = dataframe.columns.str.strip(" ")
    
    
    if "crm" in dataframe.columns:
        dataframe.drop(['crm'], axis=1, inplace=True)
    
    if 'SEPI_tools' in dataframe.columns:
        dataframe.drop(['SEPI_tools'], axis=1, inplace=True)
        
    return dataframe


df_user_matrix = get_user_data_matrix(df_user)

In [885]:
df_user_matrix

,email,mobile,annual,usa/northamerica,contactenrichment,intentdata,midmarket,hubspot
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5


In [886]:
for i,item in enumerate(df_pro):
    print(item,i)
    if item in df_user_matrix.columns:
        print(item,i)
        df_pro[item][0] = df_pro[item][0] * df_user_matrix[item][0]
    else:
        df_pro.drop(item, axis=1, inplace=True)
        
df_pro

contactenrichment 0
contactenrichment 0
intentdata 1
intentdata 1
annual 2
annual 2
monthly 3
email 4
email 4
workphone 5
mobile 6
mobile 6
microsoft dynamics 7
hubspot 8
hubspot 8
salesforce 9
pipedrive 10
sugarcrm 11
zoho 12
insightly 13
copper 14
nimble 15
infusionsoft 16
capsule 17
nutshell 18
base 19
vtiger 20
keap 21
freshsales 22
freshworks 23
outreach 24
salesloft 25
reply 26
woodpecker 27
lemlist 28
close 29
smb 30
midmarket 31
midmarket 31
enterprise  32
usa/northamerica 33
usa/northamerica 33
international 34


,contactenrichment,intentdata,annual,email,mobile,hubspot,midmarket,usa/northamerica
Company,,,,,,,,
ZoomInfo,1.0,1.0,1.0,1.0,1.0,0.5,1.0,1.0
Cognism,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
Uplead,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Lead411,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Apollo,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
Seamless,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
RocketReach,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
Lusha,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0
Crunchbase,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0


In [887]:
# for index, row in df_user.iterrows():
#     print(row)
    
    
#     if df_user['Location'].item().replace("'", "").strip("[]") == "Global":
#         df_user['Location'] = [['usa/northamerica', 'international']]
#     else:
#         df_user['Location'] = row['Location'].item().replace("'", "").strip("[]").lower()
#         df_user['Location'] = row['Location'].str.split(",")
        

#     if df_user['crm'].item().replace("\"", "").strip("[]") == "I don't have any CRM":
#         df_user.drop('crm', axis=1, inplace=True)
#     else:
#         df_user['crm'] = df_user['crm'].item().replace("'", "").strip("[]")
#         df_user['crm'] = df_user['crm'].str.split(",")
        
#     if df_user['SEPI_tools'].item().replace("\"", "").strip("[]") == "I don't have a sales engagement platform":
#         df_user.drop('SEPI_tools', axis=1, inplace=True)
#     else:
#         df_user['SEPI_tools'] = df_user['SEPI_tools'].item().replace("'", "").strip("[]")
#         df_user['SEPI_tools'] = df_user['SEPI_tools'].str.split(",")

In [888]:
# df_user['data'] = df_user['data'].item().replace("'", "").strip("[]").lower()
# df_user['data'] = df_user['data'].str.split(",")

# df_user['contract'] = df_user['contract'].item().replace("'", "").strip("[]")
# df_user['contract'] = df_user['contract'].str.split(",")

# df_user['contacts'] = df_user['contacts'].item().replace("'", "").strip("[]")

# df_user['duration'] = df_user['duration'].item().replace("'", "").strip("[]")

# if df_user['Location'].item().replace("'", "").strip("[]") == "Global":
#     df_user['Location'] = [['USA/NorthAmerica', 'International']]
# else:
#     df_user['Location'] = df_user['Location'].item().replace("'", "").strip("[]")
#     df_user['Location'] = df_user['Location'].str.split(",")
    
# df_user['features'] = df_user['features'].item().replace("'", "").strip("[]")
# df_user['features'] = df_user['features'].str.split(",")

# df_user['salesTeam'] = df_user['salesTeam'].item().replace("'", "").strip("[]")
# df_user['salesTeam'] = df_user['salesTeam'].str.split(",")

# if df_user['crm'].item().replace("\"", "").strip("[]") == "I don't have any CRM":
#     df_user.drop('crm', axis=1, inplace=True)
# else:
#     df_user['crm'] = df_user['crm'].item().replace("'", "").strip("[]")
#     df_user['crm'] = df_user['crm'].str.split(",")
    
# if df_user['SEPI_tools'].item().replace("\"", "").strip("[]") == "I don't have a sales engagement platform":
#     df_user.drop('SEPI_tools', axis=1, inplace=True)
# else:
#     df_user['SEPI_tools'] = df_user['SEPI_tools'].item().replace("'", "").strip("[]")
#     df_user['SEPI_tools'] = df_user['SEPI_tools'].str.split(",")

In [902]:
df_pro.mean()

contactenrichment    0.70
intentdata           0.40
annual               1.00
email                1.00
mobile               0.70
hubspot              0.85
midmarket            0.90
usa/northamerica     0.90
dtype: float64

In [906]:
(df_pro.sum(axis=1)/(df_pro.sum(axis=1)).sum(axis=0)).sort_values()

Company
Crunchbase     0.062016
Lusha          0.077519
Snov.io        0.077519
RocketReach    0.093023
Cognism        0.108527
Apollo         0.108527
Seamless       0.108527
ZoomInfo       0.116279
Uplead         0.124031
Lead411        0.124031
dtype: float64

In [907]:
df_soft['recommend']= df_pro.sum(axis=1)
#add back the pricing range column 
df_soft['Pricing Range'] = df['Pricing Range']


In [922]:
if "monthly" in df_user['duration'].item():
    result = df_soft[df_soft['Starting Price - Monthly'] <= df_user['price'].item()].sort_values(by='Starting Price - Monthly', ascending=True)
elif "annual" in df_user['duration'].item():
    result = df_soft[df_soft['Starting Price - Annually'] <= df_user['price'].item()].sort_values(by='Starting Price - Annually', ascending=True)
else:
    result = df_soft.sort_values(by='Starting Price - Monthly', ascending=False)

In [923]:
result

,Starting Price - Monthly,Starting Price - Annually,Contract,Solutions,CRM Integrations,SEPI,SalesTeamType,ContactEnrichment,IntentData,location,recommend,Pricing Range
Company,,,,,,,,,,,,
Apollo,49,0,Annual or Monthly,"email,workphone,mobile","Salesforce,HubSpot","Salesloft,Outreach","SMB,MidMarket,Enterprise",Yes,No,USA/NorthAmerica,7.0,"$49 per month, month to month - $6000 per year+"
Lusha,39,348,Annual or Monthly,"email,workphone","Salesforce,HubSpot","Salesloft,Outreach",SMB,Yes,No,USA/NorthAmerica,5.0,"$39 per monmth to $1,140 per year+"
Crunchbase,0,348,Annual or Monthly,email,Salesforce,-,"SMB,MidMarket,Enterprise",No,No,USA/NorthAmerica,4.0,348
Snov.io,39,396,Annual,email,"HubSpot,Salesforce,Pipedrive,Nutshell,Zoho,Fre...",-,"SMB,MidMarket",No,No,USA/NorthAmerica,5.0,"$39 per month to & 7,380"
RocketReach,80,468,Annual or Monthly,"email,workphone,mobile","Salesforce,HubSpot","Salesloft,Outreach","SMB,MidMarket",No,No,USA/NorthAmerica,6.0,$80 per month to $6000 per year+
Uplead,99,888,Annual or Monthly,"email,workphone,mobile","Salesforce,HubSpot,Insightly,Pipedrive,Zoho,Mi...","Outreach,Salesloft,Reply,Woodpecker,Lemlist,Close","SMB,MidMarket,Enterprise",Yes,Yes,USA/NorthAmerica,8.0,"$888 annual, $99 per month, month to month - ..."
Lead411,99,900,Annual or Monthly,"email,workphone,mobile","Salesforce,HubSpot,Pipedrive,Microsoft Dynamic...",-,"SMB,MidMarket,Enterprise",Yes,Yes,USA/NorthAmerica,8.0,"$99 per month, month to month-$1800+ per year ..."
